In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Specify the path to the shapefile
shapefile_path = r"C:\Users\magst\Desktop\JESSE_STUDENTWORKERS\data\NIFC_2020_WFIGS_AllPerims_20231010.shp"

# Load the shapefile
dataNIFC = gpd.read_file(shapefile_path)

# Filter the data where 'Poly_Incid' is 'cameronPeak'
cameron_peak_data_NIFC = dataNIFC[dataNIFC['poly_Incid'] == 'Cameron Peak']  # Assuming case-insensitive comparison

cameron_peak_data_NIFC.plot()
plt.title("Map of Cameron Peak Fire Perimeter")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
shapefile_path = r"C:\Users\magst\Desktop\JESSE_STUDENTWORKERS\target_nf_perimeters.shp"

# Load the shapefile
dataNasa = gpd.read_file(shapefile_path)
dataNasa.columns

cameron_peak_data_Nasa = dataNasa[dataNasa['FireName'] == 'CAMERON PEAK']  # Assuming case-insensitive comparison

cameron_peak_data_Nasa.plot(facecolor='none')
plt.title("Map of Cameron Peak Fire Perimeter")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Assuming cameron_peak_data_Nasa is a GeoDataFrame
cameron_peak_sorted = cameron_peak_data_Nasa.sort_values(by='t')

# This is the day IE 55 time steps from the start of the fire
OneDay_row = cameron_peak_sorted.iloc[1]

# Convert the first row to a GeoSeries
OneDay_row_NASA = gpd.GeoSeries(OneDay_row['geometry'])

# Plot the geometry of the first row
OneDay_row_NASA.plot()
plt.title("Map of Cameron Peak Fire Perimeter (First Row)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Convert OneDay_row_NASA to a GeoDataFrame
OneDay_row_NASA_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(OneDay_row_NASA))

# Calculate the decimal degree equivalent of 100 meters
meters = 30

degrees = meters / 111000  # conversion factor from meters to degrees

buffered_cameron_peak = cameron_peak_data_NIFC.buffer(degrees)

# Create a unary union of the buffered geometries to handle multiple polygons
buffered_union = buffered_cameron_peak.unary_union

# Check for intersections and adjust geometries
OneDay_row_NASA_gdf['adjusted_geometry'] = OneDay_row_NASA_gdf.apply(
    lambda row: row['geometry'].intersection(buffered_union) if row['geometry'].intersects(buffered_union) else row['geometry'],
    axis=1
)

# You must now remove the negative buffer
OneDay_row_NASA_gdf['Removebuffered_adjusted_geometry'] = OneDay_row_NASA_gdf['adjusted_geometry'].buffer(-degrees)


# PLOTTING 
# Plotting both the adjusted geometry and the original NIFC data
fig, ax = plt.subplots(figsize=(12, 8))  # Width is 12 inches and height is 8 inches

# plot the corrected NASA perimeter
OneDay_row_NASA_gdf['Removebuffered_adjusted_geometry'].plot(ax=ax, color='blue', label='Adjusted Perimeter')
# Plot the original NIFC data
cameron_peak_data_NIFC.plot(ax=ax, color='red', alpha=0.3, label='Original Perimeter')
# Get the total bounds of OneDay_row_NASA_gdf's adjusted geometry
bounds = OneDay_row_NASA_gdf['Removebuffered_adjusted_geometry'].total_bounds
ax.set_xlim([bounds[0], bounds[2]])  # Set x limits (min and max longitude)
ax.set_ylim([bounds[1], bounds[3]])  # Set y limits (min and max latitude)

# Add plot titles and labels
plt.title("Comparison of Original and Adjusted Perimeter of Cameron Peak Fire")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Now put it in a loop 

In [ ]:
for i in range(len(cameron_peak_sorted)):
    print(i)
    